This notebook will serve as a testbed for custom shape-targeting forward and backward development and testing.

Every cell done below would able to run independently after first import

In [16]:
# First, add a custom function to deformable.h and deformable.cpp that make sure compiles and can modify data
import sys
sys.path.append('../')
import os

from pathlib import Path
import time
import numpy as np
import scipy.optimize
import pickle
import matplotlib.pyplot as plt

from py_diff_pd.common.common import ndarray, create_folder
from py_diff_pd.common.common import print_info, print_ok, print_error, print_warning
from py_diff_pd.common.grad_check import check_gradients
from py_diff_pd.common.display import export_gif
from py_diff_pd.core.py_diff_pd_core import StdRealVector
from py_diff_pd.env.soft_starfish_env_3d import SoftStarfishEnv3d
from py_diff_pd.common.project_path import root_path
from py_diff_pd.core.py_diff_pd_core import HexMesh3d, HexDeformable, StdRealVector
import py_diff_pd.common.hex_mesh as hex

ShapeTarget Compile test

In [17]:
# initailize an array of 0, len 100
x = np.zeros(100)
x = StdRealVector(x)
y = np.zeros(100)
y = StdRealVector(y)

deformable = HexDeformable()

# const q, v, actuation, dt, pd_options, 
# writable q_next, v_next 
# Assume 3000 elements, 5000 vertices 
dt = 0.01
dof  =200
options = {
    'max_pd_iter': 1000,
    'thread_ct': 20,
    'abs_tol': 1e-6,
    'rel_tol': 1e-6,
    'verbose': 0,
    'use_bfgs': 1,
    'bfgs_history_size': 10,
    'max_ls_iter': 10,
    
}
q, v, act =  StdRealVector(dof), StdRealVector(dof), StdRealVector(dof)
q_next, v_next = StdRealVector(dof), StdRealVector(dof)
deformable.PyShapeTargetingForward( q, v, act, dt, options, q_next, v_next)
q_next = np.array(q_next)
# avg diff between q and q_next
print(np.mean(np.abs(q_next - q)))



q_next: 0 0 0 0 0 0 0 0 0 0
0.0


example of getting a ShapeTarget act matrix given target q

In [18]:
# Load the json output from soft_starfish.ipynb, 
# which output the ideal pos for each hex verts.
# Then we can compute the Deformation gradient, and take the R
# R stored in act, size is num_of_elements * (sample_num * 6), where 6 is the symmetric part of R

obj_num = 90 

# common parameters
asset_folder = Path('/mnt/e/muscleCode/sample_muscle_data/starfish')
mesh_bin_str = str(asset_folder / 'starfish_demo_voxel.bin')
gt_folder = Path('/mnt/e/wsl_projects/diff_pd_public/python/example/quasi_starfish/init_ground_truth')
gt_json = str(gt_folder)+ '/'+ 'starfish_' + str(obj_num) + '_init_ground_truth.json'

render_folder = Path('/mnt/e/wsl_projects/diff_pd_public/python/example/quasi_starfish')
render_bin_path = render_folder / str('starfish_voxel_'+str(obj_num)+'.bin')
render_bin_str = str(render_bin_path)
obj_output_file = render_folder / str('starfish_voxel_'+str(obj_num)+'.obj')
obj_output_str = str(obj_output_file)
# pass whole file into a list
float_list = []
with open(gt_json, 'r') as f:
    content = f.read()
    float_strings = content.strip('[]').split(',')
    float_list = [float(x) for x in float_strings]
# Deformable param
youngs_modulus = 5e5
poissons_ratio = 0.45
la = youngs_modulus * poissons_ratio / ((1 + poissons_ratio) * (1 - 2 * poissons_ratio))
mu = youngs_modulus / (2 * (1 + poissons_ratio))
density = 1e3
thread_ct = 20
dt = 1e-2

default_hex_mesh = HexMesh3d()
default_hex_mesh.Initialize(mesh_bin_str) 
deformable = HexDeformable()
deformable.Initialize(mesh_bin_str, density, 'none', youngs_modulus, poissons_ratio)

act = StdRealVector(0)
deformable.PyGetShapeTargetSMatrixFromDeformation(float_list, act)
act = np.array(act)
print(int(act.shape[0] // 48) == default_hex_mesh.NumOfElements())


True


TypeError: Wrong number or type of arguments for overloaded function 'new_StdRealVector'.
  Possible C/C++ prototypes are:
    std::vector< real >::vector()
    std::vector< real >::vector(std::vector< real > const &)
    std::vector< real >::vector(std::vector< double >::size_type)
    std::vector< real >::vector(std::vector< double >::size_type,std::vector< double >::value_type const &)


Next we input the act into the forward pass and compare the diff, after some number of iterations

In [ ]:
  # visualize the hex mesh
from py_diff_pd.common.renderer import PbrtRenderer
png_file = render_folder / 'starfish_default.png'
def render_quasi_starfish(mesh_file, png_file):
    options = {
        'file_name': png_file,
        'light_map': 'uffizi-large.exr',
        'sample': 4,
        'max_depth': 2,
        'camera_pos': (20, 30, 50),
        'camera_lookat': (1, -1, 0), # roughly the center of starfish obj
        
    }
    renderer = PbrtRenderer(options)
    
    mesh = HexMesh3d()
    mesh.Initialize(mesh_file)
    renderer.add_hex_mesh(mesh, render_voxel_edge=True, color=(.3, .7, .5), transforms=[
        ('r', [90, 1, 0, 0]),  # Rotate 90 degrees around the x-axis
        ('t', [0, 0, 0]),
        ])
    # renderer.add_tri_mesh(Path(root_path) / 'asset/mesh/flat_ground.obj',
    #         texture_img='chkbd_24_0.7', transforms=[
    #             ('s', 4),
    #             ('t', [0, 0, -1]),
    #             ])
    
    
    renderer.render()



In [ ]:
deformable_shapeTarget = HexDeformable()
deformable_shapeTarget.Initialize(mesh_bin_str, density, 'none', youngs_modulus, poissons_ratio)

png_file = render_folder / f'starfish_{obj_num}_shape_target.png'

q_curr = default_hex_mesh.py_vertices()
v_curr = np.zeros(deformable_shapeTarget.dofs())
q_next, v_next = StdRealVector(dof), StdRealVector(dof) 

deformable_shapeTarget.PySaveToMeshFile(q_curr, render_bin_str)
render_quasi_starfish(render_bin_str, png_file)

In [ ]:
deformable_shapeTarget.PyShapeTargetingForward(q_curr, v_curr, act, dt, options, q_next, v_next)
q_next = np.array(q_next)
deformable_shapeTarget.PySaveToMeshFile(q_next, render_bin_str)
png_file = render_folder / f'starfish_{obj_num}_shape_target_next.png'
render_quasi_starfish(render_bin_str, png_file)



TypeError: in method 'Deformable38_PyShapeTargetingForward', argument 4 of type 'std::vector< real,std::allocator< real > > const &'

In [ ]:
# print avg diff between q and q_next
print(np.mean(np.abs(q_next - q_curr)))

In [ ]:
v_next = np.array(v_next)
q_next, v_next